# Technical Indicators with **Polar TI*** This is a **Work in Progress** and subject to change!
* Contributions are welcome and accepted!
* Examples below are for **educational purposes only**.
* **NOTE:** The **watchlist** module is independent of Polars TI. To easily use it, copy it from your local polars-ti installation directory into your project directory.

### Required Packages
##### Uncomment the packages you need to install or are missing

In [1]:
#!pip install numpy
#!pip install pandas
#!pip install mplfinance
#!pip install pandas-datareader
#!pip install requests_cache
#!pip install tqdm
#!pip install alphaVantage-api # Required for Watchlist

In [ ]:
%pylab inline
from datetime import datetime
import random as rnd
from sys import float_info as sflt

from tqdm import tqdm

import numpy as np
import pandas as pd

# pd.set_option("max_rows", 100)
# pd.set_option("max_columns", 20)

import mplfinance as mpf
import polars_ti as ti

from tqdm.notebook import trange, tqdm

from watchlist import (
    colors,
    Watchlist,
)  # Is this failing? If so, copy it locally. See above.

print(f"Numpy v{np.__version__}")
print(f"Pandas v{pd.__version__}")
print(f"mplfinance v{mpf.__version__}")
print(
    f"\nPolars TI v{ti.version}\nTo install the Latest Version:\n$ pip install -U git+https://github.com/CMobley7/polars-ti\n"
)
%matplotlib inline

### MISC Function(s)

In [ ]:
def recent_bars(df, tf: str = "1y"):
    # All Data: 0, Last Four Years: 0.25, Last Two Years: 0.5, This Year: 1, Last Half Year: 2, Last Quarter: 4
    yearly_divisor = {
        "all": 0,
        "10y": 0.1,
        "5y": 0.2,
        "4y": 0.25,
        "3y": 1.0 / 3,
        "2y": 0.5,
        "1y": 1,
        "6mo": 2,
        "3mo": 4,
    }
    yd = yearly_divisor[tf] if tf in yearly_divisor.keys() else 0
    return int(ti.RATE["TRADING_DAYS_PER_YEAR"] / yd) if yd > 0 else df.shape[0]

## Data Collection

In [ ]:
tf = "D"
tickers = ["SPY", "QQQ", "AAPL", "TSLA", "BTC-USD"]
watch = Watchlist(tickers, tf=tf, ds_name="yahoo", timed=True)
# watch.study = ti.CommonStudy # If you have a Custom Study, you can use it here.
watch.load(tickers, analyze=True, verbose=False)

# Asset Selection

In [ ]:
ticker = tickers[1]  # change tickers by changing the index
print(
    f"{ticker} {watch.data[ticker].shape}\nColumns: {', '.join(list(watch.data[ticker].columns))}"
)

### Trim it

In [ ]:
duration = "5y"
asset = watch.data[ticker]
recent = recent_bars(asset, duration)
asset.columns = asset.columns.str.lower()
asset.drop(columns=["dividends", "split"], errors="ignore", inplace=True)
asset = asset.copy().tail(recent)
asset

# Trend Creation
A **Trend** is the result of some calculation or condition of one or more indicators. For simplicity, a _Trend_ is either ```True``` or ```1``` and _No Trend_ is ```False``` or ```0```. Using the **Hello World** of Trends, the **Golden/Death Cross**, it's Trend is _Long_ when ```long = ma(close, 50) > ma(close, 200) ``` and _Short_ when ```short = ma(close, 50) < ma(close, 200) ```. 

In [ ]:
# Example Long Trends
# long = ti.sma(asset.close, 50) > ti.sma(asset.close, 200) # SMA(50) > SMA(200) "Golden/Death Cross"
long = ti.sma(asset.close, 20) > ti.sma(asset.close, 50)  # SMA(20) > SMA(50)
# long = ti.ema(asset.close, 8) > ti.ema(asset.close, 21) # EMA(8) > EMA(21)
# long = ti.increasing(ti.ema(asset.close, 20))
# long = ti.macd(asset.close).iloc[:,1] > 0 # MACD Histogram is positive

# long &= ti.increasing(ti.ema(asset.close, 50), 2) # Uncomment for further long restrictions, in this case when EMA(50) is increasing/sloping upwards
# long = 1 - long # uncomment to create a short signal of the trend

asset.ti.ema(length=8, sma=False, append=True)
asset.ti.ema(length=21, sma=False, append=True)
asset.ti.ema(length=50, sma=False, append=True)
asset.ti.percent_return(append=True, cumulative=False)
print("TI Columns Added:")
asset[asset.columns[5:]].tail()

### **Trend Signals** 
Given a _Trend_, **Trend Signals** returns the _Trend_, _Trades_, _Entries_ and _Exits_ as boolean integers. When ```asbool=True```, it returns _Trends_, _Entries_ and _Exits_ as boolean values which is helpful when combined with the [**vectorbt**](https://github.com/polakowo/vectorbt) backtesting package.

In [ ]:
trendy = asset.ti.tsignals(long, asbool=False, append=True)
trendy.tail()

### Trend Entries & Exits & Trade Table
This is a simple way to reduce the Asset DataFrame to a Trade Table with Dates, Signals, and Entries and Exits. Gives you an idea what to expect before running through a backtester such as [**vectorbt**](https://github.com/polakowo/vectorbt).

In [ ]:
entries = trendy.TS_Entries * asset.close
entries = entries[~np.isclose(entries, 0)]
entries.dropna(inplace=True)
entries.name = "Entry"

exits = trendy.TS_Exits * asset.close
exits = exits[~np.isclose(exits, 0)]
exits.dropna(inplace=True)
exits.name = "Exit"

total_trades = trendy.TS_Trades.abs().sum()
rt_trades = int(trendy.TS_Trades.abs().sum() // 2)

all_trades = trendy.TS_Trades.copy().fillna(0)
all_trades = all_trades[all_trades != 0]

trades = pd.DataFrame({"Signal": all_trades, entries.name: entries, exits.name: exits})

# Show some stats if there is an active trade (when there is an odd number of round trip trades)
if total_trades % 2 != 0:
    unrealized_pnl = asset.close.iloc[-1] - entries.iloc[-1]
    unrealized_pnl_pct_change = 100 * ((asset.close.iloc[-1] / entries.iloc[-1]) - 1)
    print("Current Trade:")
    print(f"Price Entry | Last:\t{entries.iloc[-1]:.4f} | {asset.close.iloc[-1]:.4f}")
    print(
        f"Unrealized PnL | %:\t{unrealized_pnl:.4f} | {unrealized_pnl_pct_change:.4f}%"
    )
print(f"\nTrades Total | Round Trip:\t{total_trades} | {rt_trades}")
print(f"Trade Coverage: {100 * asset.TS_Trends.sum() / asset.shape[0]:.2f}%")

tradelist = trades
if rt_trades > 10:
    tradelist = trades.tail(10)
tradelist

# Visualization

### Chart Display Strings

In [ ]:
extime = ti.get_time(to_string=True)
first_date, last_date = asset.index[0], asset.index[-1]
last_ohlcv = f"Last OHLCV: ({asset.iloc[-1].open:.4f}, {asset.iloc[-1].high:.4f}, {asset.iloc[-1].low:.4f}, {asset.iloc[-1].close:.4f}, {int(asset.iloc[-1].volume)})"
_oc_change = asset.iloc[-1].open - asset.iloc[-1].close
_oc_change_pct = _oc_change / asset.iloc[-1].open
oc_change = f"{_oc_change:.4f} ({100 * _oc_change_pct:.4f} %)"
ptitle = f"\n{ticker} [{tf} for {duration}({recent} bars)]\n{last_ohlcv}, Change (%): {oc_change}\n{extime}"

### Trade Chart

In [ ]:
# chart = asset["close"] #asset[["close", "SMA_10", "SMA_20", "SMA_50", "SMA_200"]]
# chart = asset[["close", "SMA_10", "SMA_20"]]
chart = asset[["close", "EMA_8", "EMA_21", "EMA_50"]]
chart.plot(figsize=(16, 10), color=colors("BkGrOrRd"), title=ptitle, grid=True)

### Long and Short Trends
**Trends** are either a _Trend_ (```1```) or _No Trend_ (```0```) depending on the **Trend** passed into ***Trend Signals**

In [ ]:
long_trend = trendy.TS_Trends
short_trend = 1 - long_trend

long_trend.plot(
    figsize=(16, 0.85), kind="area", stacked=True, color=["limegreen"], alpha=0.65
)  # Green Area
short_trend.plot(
    figsize=(16, 0.85), kind="area", stacked=True, color=["orangered"], alpha=0.65
)  # Red Area

### Trades or Trade Signals
The **Trades** are either _Enter_ (```1```) or _Exit_ (```-1```) or _No Position/Action_ (```0```). These are based on the **Trend** passed into **Trend Signals** whether they are _Long_ or _Short_ Trends.

In [ ]:
trendy.TS_Trades.plot(figsize=(16, 1.5), color=colors("BkBl")[0], grid=True)

### Active Returns
**Active Returns** are returns made during the course of the _Trend_. They are simply the product of the returns and the _Trend_

In [ ]:
asset["ACTRET_1"] = trendy.TS_Trends.shift(1) * asset.PCTRET_1
asset[["PCTRET_1", "ACTRET_1"]].plot(
    figsize=(16, 3), color=["gray", "limegreen"], alpha=1, grid=True
).axhline(0, color="black")

### Buy and Hold Returns (*PCTRET_1*)

In [ ]:
((asset.PCTRET_1 + 1).cumprod() - 1).plot(
    figsize=(16, 3),
    kind="area",
    stacked=False,
    color=["limegreen"],
    title="B&H Percent Returns",
    alpha=0.9,
    grid=True,
).axhline(0, color="black")

### Cum. Active Returns (*ACTRET_1*)

In [ ]:
# ((asset.ACTRET_1 + 1).cumprod() - 1).plot(figsize=(16, 3), kind="area", stacked=False, color=colors("GyOr")[-1], title="B&H Cum. Active Returns", alpha=0.4, grid=True).axhline(0, color="black")
((asset.ACTRET_1 + 1).cumprod() - 1).plot(
    figsize=(16, 3),
    kind="area",
    stacked=False,
    color=["limegreen"],
    title="B&H Cum. Active Returns",
    alpha=0.65,
    grid=True,
).axhline(0, color="black")

# Disclaimer
* All investments involve risk, and the past performance of a security, industry, sector, market, financial product, trading strategy, or individual’s trading does not guarantee future results or returns. Investors are fully responsible for any investment decisions they make. Such decisions should be based solely on an evaluation of their financial circumstances, investment objectives, risk tolerance, and liquidity needs.

* Any opinions, news, research, analyses, prices, or other information offered is provided as general market commentary, and does not constitute investment advice. I will not accept liability for any loss or damage, including without limitation any loss of profit, which may arise directly or indirectly from use of or reliance on such information.